In [1]:
import os
import sys

import numpy as np

sys.path.append(r"C:\Users\zschende\OneDrive - Ilmatieteen laitos\Data\smrt")

In [2]:
# import smrt
# from smrt.atmosphere.simple_atmosphere import SimpleAtmosphere
from smrt import make_atmosphere, make_model, make_snowpack, make_soil, sensor_list
from smrt.core.globalconstants import PERMITTIVITY_OF_AIR

In [3]:
# CIMR as passive microwave sensor
sensor = sensor_list.cimr(channel=["18", "37"])  # add theta=[53, 55] for feeds

Transmissivity

In [4]:
def extract_configuration(transmissivity_map):
    keys = ["frequency", "polarization", "theta", "transmissivity"]

    configuration = dict()
    for k in keys:
        try:
            x = np.unique([transmissivity_map[ch][k] for ch in transmissivity_map])
            if len(x) == 1:
                x = x[0]
            configuration[k] = x
        except KeyError:
            continue

    return configuration

In [5]:
# TODO veg: make_atmosphere needs vol as input
# use frequency, polarization, theta from channel_map


# from smrt.core.layer import layer_properties
# @layer_properties("stem_volume", "ke")  # may not work, better use **kwargs like make_soil
def forest_transmissivity_cohen(frequency, theta, mode, stem_volume=50, ke=None):
    """Calculate one-way transmissivity for forest canopy.

    _extended_summary_
    transmissivity (considered equal to transmittance in this case)

    Args:
        frequency (_type_): _description_
        theta (_type_): _description_
        vol (_type_): Stem volume in m3/ha
        ke (_type_, optional): Extinction coefficient. Defaults to None.

    Returns:
        _type_: _description_
    """

    costheta = np.cos(theta)

    # frequency_dict = {"%02i" % (frequency): frequency for freq in np.atleast_1d(frequency)}

    # passive mode
    if mode == "P":
        polarization = ["V", "H"]

        if ke is None:
            if frequency < 17e9:
                ke = np.array([0, 0])  # negligible attenuation
            elif frequency > 17e9 and frequency < 20e9:
                ke = np.array([0.007, 0.010])
            elif frequency > 35e9 and frequency < 39e9:
                ke = np.array([0.011, 0.012])
            else:
                raise ValueError("Frequency not supported")

        if theta != np.deg2rad(50):
            ke = [k * np.cos(np.deg2rad(50)) for k in ke]

        transmissivity = np.exp((-ke * stem_volume) / costheta)

    # active mode not yet implemented
    elif mode == "A":
        pass

    transmissivity_map = {
        "%02i" % (frequency / 1e9) + pola: dict(
            frequency=frequency,
            polarization=pola,
            theta=np.rad2deg(theta),
            transmissivity=transmissivity[idpola],
        )
        for idpola, pola in enumerate(polarization)
    }

    return transmissivity_map, transmissivity

In [6]:
transmissivity_map, transmissivity = forest_transmissivity_cohen(
    frequency=sensor.frequency[1], theta=np.deg2rad(50), mode="P", stem_volume=10
)

In [7]:
transmissivity

array([0.84271233, 0.82970351])

In [8]:
transmissivity_map

{'36V': {'frequency': np.float64(36500000000.0),
  'polarization': 'V',
  'theta': np.float64(50.0),
  'transmissivity': np.float64(0.842712333214737)},
 '36H': {'frequency': np.float64(36500000000.0),
  'polarization': 'H',
  'theta': np.float64(50.0),
  'transmissivity': np.float64(0.8297035097119744)}}

In [9]:
extract_configuration(transmissivity_map)

{'frequency': np.float64(36500000000.0),
 'polarization': array(['H', 'V'], dtype='<U1'),
 'theta': np.float64(50.0),
 'transmissivity': array([0.82970351, 0.84271233])}

Atmosphere

In [10]:
atmos_iso = make_atmosphere("simple_isotropic_atmosphere", tb_down=23, tb_up=23, transmittance=0.5)

In [12]:
atmos = make_atmosphere(
    "simple_atmosphere",
    theta=[0, 20, 40],
    tb_down={18.7e9: [23, 58, 64], 37e9: [10, 20, 30]},
    tb_up={18.7e9: [5, 10, 15], 37e9: [6, 7, 8]},
    transmittance={18.7e9: [1, 0.9], 37e9: [0.95, 0.85, 0.75]},
)

SMRTError: The length of the transmittance values must match the length of the theta array. Got 2 values for 3 angles.

In [16]:
atmos.run(frequency=18e9, costheta=np.cos(np.deg2rad(50)), npol=2)

SMRTError: Frequency 18000000000.0 not found in atmosphere parameters.

In [13]:
# Set frozen soil as substrate with values from Meloche et al. (2021)
epsr = 3.3
epsi = 0.0051
rms = 1.65e-2
tsoil = 270

sub = make_soil(
    "soil_wegmuller",
    permittivity_model=complex(epsr, epsi),
    roughness_rms=rms,
    temperature=tsoil,
)

In [ ]:
# Set synthetic truth snowpack properties to obtain synthetic TB_obs
sp = make_snowpack(
    thickness=np.array([1]),
    microstructure_model="exponential",
    density=[300],
    temperature=[265],
    ice_permittivity_model=None,
    background_permittivity_model=PERMITTIVITY_OF_AIR,
    liquid_water=0,
    salinity=0,
    corr_length=[1 * 4 * (1 - 300 / 917) / 917 / 25],
    substrate=sub,
    atmosphere=atmos_iso,
)

In [15]:
# Initialize the SMRT model with IBA and DORT solver
smrt_model = make_model(
    "iba",
    "dort",
    rtsolver_options={
        "error_handling": "nan",
        "phase_normalization": True,
        "diagonalization_method": "shur_forcedtriu",
    },
)

In [17]:
# Synthetic TB observations
TB_obs = smrt_model.run(sensor, sp)

KeyError: np.float64(18700000000.0)

In [ ]:
# no atmosphere
TB_obs.TbH()

<xarray.DataArray (frequency: 2)> Size: 16B
array([250.61961632, 236.48431851])
Coordinates:
    theta         float64 8B 55.0
    polarization  <U1 4B 'H'
  * frequency     (frequency) float64 16B 1.87e+10 3.65e+10
Attributes:
    mode:     P

In [ ]:
# no atmosphere
TB_obs.TbV()

<xarray.DataArray (frequency: 2)> Size: 16B
array([265.68503605, 253.71498309])
Coordinates:
    theta         float64 8B 55.0
    polarization  <U1 4B 'V'
  * frequency     (frequency) float64 16B 1.87e+10 3.65e+10
Attributes:
    mode:     P

In [ ]:
# simple atmosphere
TB_obs.TbH()

<xarray.DataArray (frequency: 2)> Size: 16B
array([219.04767754, 187.8355027 ])
Coordinates:
    theta         float64 8B 55.0
    polarization  <U1 4B 'H'
  * frequency     (frequency) float64 16B 1.87e+10 3.65e+10
Attributes:
    mode:     P

In [ ]:
# simple atmosphere
TB_obs.TbV()

<xarray.DataArray (frequency: 2)> Size: 16B
array([228.24337575, 199.35822274])
Coordinates:
    theta         float64 8B 55.0
    polarization  <U1 4B 'V'
  * frequency     (frequency) float64 16B 1.87e+10 3.65e+10
Attributes:
    mode:     P